#### (4月12日) Corpus Analysis

In [83]:
import numpy as np
import pandas as pd
import copy
import re
from tqdm import tqdm 

pd.options.mode.chained_assignment = None

In [2]:
# Unpickle
corpus_new = pd.read_pickle('savefiles/corpusfull_20220410.pkl')

In [3]:
def collapse(text):
    
    """
    Paragraphs extracted from XML files contain different numbers of sentences,
    and even incomplete sentences. This function collapses a list of paragraphs
    into a list of sentences or sentence-equivalents in advance of NLP processing.
    """
    
    doc_raw = ""
    punc = set([".", ",", ";", ":"])
    for i, para in enumerate(text):
        if para[-1] in punc:
            doc_raw += (para + " ")
        else:
            doc_raw += (para + ". ")
        
    return doc_raw

In [4]:
def preprocess_gen(text):
    
    """
    This function is primarily for removing references, and fixing spacing
    between/within sentences in preparation for scispaCy language modeling.
    """
    
    text = text.replace('\u200a', '').replace('\n', '')   # remove weird space code, newlines
    text = re.sub('\[(\d+)\]', '', text)                  # remove refs ([1], [23], etc.)
    text = text.replace(' ,', '').replace(' .', '.')      # fix spaces created by prev line
    text = text.replace(' ;', ';').replace(' :', ':')     # 
    text = text.replace('  ', ' ')
    
    return text

In [5]:
# Converts each doc from list of paras to one long string
for i in range(0,20):
    corpus_new.loc[i, "Text"] = collapse(corpus_new.loc[i, "Text"])

# Preprocess each doc before Spacy modeling
for i in range(0,20):
    corpus_new.loc[i, "Text"] = preprocess_gen(corpus_new.loc[i, "Text"])
    
# For practicing on single texts
# text = corpus_new.loc[0, "Text"]
# text

In [74]:
# Row indices of docs in respective corpora
jpen_i = [0,  1,  5, 10, 11, 12, 13, 14, 17, 19]
enen_i = [2,  3,  4,  6,  7,  8,  9, 15, 16, 18]

In [6]:
def initialize_results_df():
    
    """
    Results DF should be reset for every hypothesis tested.
    Note: Run BEFORE preprocess_tok to capture unanalyzed tokens.
    """
    
    results_df = corpus_new[['Group', 'Author', 'Title']]
    results_df['Title'] = pd.Series([title[:31] for title in results_df['Title']])  # prune title for readability
    
    # with word count
    for i in range(0,20):
        # Pull text from df
        text = corpus_new.loc[i, "Text"]
        # Split by word (" " for simplicity)
        word_ct = len(text.split(' '))
        # Put in results_df
        results_df.loc[i, 'Word Count'] = word_ct
    
    return results_df

#### Language modeling

In [7]:
import scispacy
import spacy

nlp = spacy.load("en_core_sci_md")

In [8]:
def unique(ls):
    
    """
    Creates a list of unique items from an existing list.
    """
 
    unique_list = []
     
    for x in ls:
        if x not in unique_list:
            unique_list.append(x)
    
    return unique_list

In [66]:
def preprocess_tok(doc):
    
    # Collect lemmas not tagged by spaCy as 1. punctuation, 2. digits, 3. URLs, or 4. stop words
    tokens = [tok.lemma_ for tok in doc if not (tok.is_punct | tok.is_digit | tok.like_url | tok.is_stop)]
    
    # Remove any tokens containing mid-string digits (e.g. "P5-a") or punc ('t(are')
    tokens = [tok for tok in tokens if not re.search("\d", tok)]
    tokens = [tok for tok in tokens if not re.search("\(", tok)]
    tokens = [tok for tok in tokens if not re.search("\)", tok)]
    
    # (4.13) Break apart hyphen- or slash-separated compounds
    seps = ['-', '―', ';', '\]', '\[', '’', '”']
    for sep in seps:
        new_toks = []
        for tok in tokens:
            new_toks += tok.split(sep)
        tokens = new_toks
    # 5. ']', '’]', '’>', '”>'
    # 6. ';.'
    # 7. ';'
#     new_toks1, new_toks2, new_toks3 = [], [], []
#     for tok in tokens: 
#         new_toks1 += tok.split('-')
#     for tok in new_toks1:
#         new_toks2 += tok.split('/')
#     for tok in new_toks2:
#         new_toks3 += tok.split('―')
#     tokens = new_toks3
    
    # (4.13) Remove remaining abbreviations
    tokens = [tok for tok in tokens if not re.search("[a-zA-Z]\.[a-zA-Z]\.", tok)]
    tokens = [tok for tok in tokens if not re.search("\+", tok)]
    
    # Remove punc and small words (e.g. 'a', 'P', 'mm')
    punc_to_skip = set(['±', '=', '>', '<'])
    tokens = [tok for tok in tokens if tok not in punc_to_skip]     # can skip?
    tokens = [tok for tok in tokens if len(tok) > 3]    
       
    # Unify to lowercase (to simplify matching)
    tokens = [tok.lower() for tok in tokens]
    
    return tokens

In [61]:
def find_tokens_unique_to_one_doc(i):
    
    other_docs = list(range(0,20))
    other_docs.remove(i)
    
    # Doc in question
    text_i = corpus_new.loc[i, "Text"]
    doc_i = nlp(text_i)
    tokens_i = unique(preprocess_tok(doc_i))
    
    # Iterate thru all 19 other docs
    for j in other_docs:
        text_j = corpus_new.loc[j, "Text"]
        doc_j = nlp(text_j)
        tokens_j = set(unique(preprocess_tok(doc_j)))
        
        for tok in tokens_i:
            if tok in tokens_j:
                tokens_i.remove(tok)
    
    return tokens_i            

In [73]:
toks_unique = find_tokens_unique_to_one_doc(11)
toks_unique

['atypical',
 'resting',
 'clustering',
 'hubness',
 'heterogeneous',
 'settle',
 'disorders',
 'edition',
 'externally',
 'under',
 'connectomic',
 'neuropathological',
 'edge',
 'nodal',
 'neurotypical',
 'backbone',
 'neurodevelopment',
 'rudie',
 'segregation',
 'barttfeld',
 'worldness',
 'neurodevelopmental',
 'accelerate',
 'trajectory',
 'anomaly',
 'pediatric',
 'aberrant',
 'weakness',
 'forty',
 'outpatient',
 'karasuyama',
 'electroconvulsive',
 'experienced',
 'infancy',
 'informant',
 'consultation',
 'hyperactivity',
 'acquaintance',
 'wechsler',
 'wais',
 'revise',
 'jart',
 'speaking',
 'print',
 'logographic',
 'antidepressant',
 'antipsychotic',
 'antiepileptic',
 'showa',
 'comorbiditie',
 'guardians',
 'doctor',
 'arbitrary',
 'anonymously',
 'locally',
 'spgr',
 'stay',
 'equation',
 'stand',
 'reconstruct',
 'skull',
 'tripped',
 'realigned',
 'stripped',
 'analytic',
 'artifactual',
 'compcor',
 'extracted',
 'spurious',
 'systematic',
 'millimeter',
 'scrubbing

In [ ]:
# Collect all tokens found in 2+ docs in JP-EN

tokens_shared_jpen = []

for i in tqdm(jpen_i):
    text_i = corpus_new.loc[i, "Text"]
    doc_i = nlp(text_i)
    tokens_i = preprocess_tok(doc_i)
    tokens_i_unique = find_tokens_unique_to_one_doc(i)
    tokens_shared_i = [tok for tok in tokens_i if tok not in set(tokens_i_unique)]
    for tok in tokens_shared_i:
        tokens_shared_jpen.append(tok)

tokens_shared_jpen_unique = unique(tokens_shared_jpen) # remove duplicates

print(f'Non-document-specific tokens in JP-EN: {len(tokens_shared_jpen_unique)}')

In [85]:
# Collect all tokens found in 2+ docs in EN-EN

tokens_shared_enen = []

for i in tqdm(enen_i):
    text_i = corpus_new.loc[i, "Text"]
    doc_i = nlp(text_i)
    tokens_i = preprocess_tok(doc_i)
    tokens_i_unique = find_tokens_unique_to_one_doc(i)
    tokens_shared_i = [tok for tok in tokens_i if tok not in set(tokens_i_unique)]
    for tok in tokens_shared_i:
        tokens_shared_enen.append(tok)

tokens_shared_enen_unique = unique(tokens_shared_enen)
print(f'Non-document-specific tokens in EN-EN: {len(tokens_shared_enen_unique)}')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [07:37<00:00, 45.75s/it]


Non-document-specific tokens in EN-EN: 2087


In [87]:
# Find intersection of tokens appearing in ≧4 docs (≧2 in each corpus) 

toks_to_analyze = [tok for tok in tokens_shared_jpen_unique if tok in set(tokens_shared_enen_unique)]
print(f'{len(toks_to_analyze)} tokens to analyze')

1848 tokens to analyze


In [89]:
# Big Kahuna

from scipy.stats import ttest_ind

docs_df = initialize_results_df()
toks_df = pd.DataFrame()

for tok in tqdm(toks_to_analyze):
    for i in range(0,20):
        text = corpus_new.loc[i, "Text"]
        doc = nlp(text)
        toks_in_doc = preprocess_tok(doc)
        counter = 0
        for doctok in toks_in_doc:
            if str(doctok) == str(tok):
                counter += 1
        docs_df.loc[i, "count"] = counter
        docs_df.loc[i, "count_adj"] = (counter / docs_df.loc[i, 'Word Count']) * 1000  # per 1000 words
    
    # Separate values by group (JP-EN v. EN-EN)
    jp_ct    = list(docs_df[docs_df['Group'] == 'JP-EN'].loc[:, "count"])
    jp_ctadj = list(docs_df[docs_df['Group'] == 'JP-EN'].loc[:, "count_adj"])
    en_ct    = list(docs_df[docs_df['Group'] == 'EN-EN'].loc[:, "count"])
    en_ctadj = list(docs_df[docs_df['Group'] == 'EN-EN'].loc[:, "count_adj"])
    
    # Calculate means and run t-tests
    jp_ct_mean    = np.mean(jp_ct)
    jp_ctadj_mean = np.mean(jp_ctadj)
    en_ct_mean    = np.mean(en_ct)
    en_ctadj_mean = np.mean(en_ctadj)
        
    P_ct    = ttest_ind(jp_ct, en_ct).pvalue
    P_ctadj = ttest_ind(jp_ctadj, en_ctadj).pvalue
    
    # Add to DF
    entry = {'Token': tok,
             'JP-EN Mean Count': jp_ct_mean, 'JP-EN Mean Freq': jp_ctadj_mean,
             'EN-EN Mean Count': en_ct_mean, 'EN-EN Mean Freq': en_ctadj_mean,
             'P (count)': P_ct, 'P (freq)': P_ctadj}
    toks_df = toks_df.append(entry, ignore_index = True)    

  0%|                                                                              | 2/1848 [01:56<29:53:30, 58.29s/it]


KeyboardInterrupt: 

In [ ]:
toks_df.to_pickle('savefiles/toksdf_20220413.pkl')

In [21]:
# Sandbox

new_words1, new_words2 = [], []
words = ['gyrus', 'parietal-temporal-occipital', 'anxiety/depression']
for word in words:
    new_words += word.split('-')

new_words

['gyrus',
 'gyrus',
 'parietal',
 'temporal',
 'occipital',
 'parietal-temporal-occipital',
 'anxiety/depression',
 'anxiety',
 'depression']

### For counting 1-grams

In [ ]:
def lexeme_counter(doc, string):
    
    """
    Function for getting raw lemma count in a document.
    """
        
    tokens = preprocess_tok(doc)
    counter = 0
    for tok in tokens:
        if str(tok) == str(string):
            counter += 1
    
    return counter

In [ ]:
results_df = initialize_results_df()

for lexeme in ["think", "consider", "report"]:
    
    new_name_n = "n_" + str(lexeme)
    new_name_adj = str(new_name_n) + "_adj"
    
    for i in range(0,20):
        # Pull text from df
        text = corpus_new.loc[i, "Text"]
        # Run scispaCy model
        doc = nlp(text)
        # Count lexeme (includes preprocessing)
        ct = lexeme_counter(doc, lexeme)
        # Put in results_df
        results_df.loc[i, new_name_n] = ct
        results_df.loc[i, new_name_adj] = (ct / results_df.loc[i, 'Word Count']) #* np.mean(results_df['Word Count'])

results_df

In [ ]:
compare_means('n_think')
compare_means('n_consider')
compare_means('n_think_adj')
compare_means('n_consider_adj')
compare_means('n_report')
compare_means('n_report_adj')

In [ ]:
for i in range(0,20):
    results_df.loc[i, 'Combined'] = results_df.loc[i, 'n_think'] + results_df.loc[i, 'n_consider']
    results_df.loc[i, 'Combined_adj'] = results_df.loc[i, 'Combined'] / results_df.loc[i, 'Word Count']
    
compare_means('Combined')
compare_means('Combined_adj')

### For 2,3-grams

In [ ]:
results_df = initialize_results_df()
cols = []

for lexeme in ["in this study", "therefore", "in addition", "bold response"]:
    
    new_name_n = "n_" + str(lexeme.replace(" ", ""))
    new_name_adj = str(new_name_n) + "_adj"
    cols.append(new_name_n)
    cols.append(new_name_adj)

    for i in range(0,20):
        text = corpus_new.loc[i, "Text"].lower()
        ct = text.count(lexeme)
        # Put in results_df
        results_df.loc[i, new_name_n] = ct
        results_df.loc[i, new_name_adj] = (ct / results_df.loc[i, 'Word Count']) #* np.mean(results_df['Word Count'])

results_df

In [ ]:
for col in cols:
    compare_means(col)

### Statistical testing
#### 1. Type/token ratio (lexical diversity)

In [ ]:
def type_token_ratio(doc):
    
    token_list = preprocess_tok(doc)
    n_type = len(unique(token_list))
    n_token = len(token_list)
    ttr = n_type/n_token
    
    return ttr

In [11]:
def compare_means(var):
    
    from scipy.stats import ttest_ind
    
    jp_stats = list(results_df[results_df['Group'] == 'JP-EN'].loc[:, var])
    en_stats = list(results_df[results_df['Group'] == 'EN-EN'].loc[:, var])
    P = ttest_ind(jp_stats, en_stats).pvalue
    
    print(f'Mean {var}, JP-EN:  {np.mean(jp_stats)}')
    print(f'Mean {var}, EN-EN:  {np.mean(en_stats)}')
    print(f'Sig. (unpaired t-test): {P}')
    print('\n')

In [ ]:
# Calculate raw type/token ratio (lemmatized)

for i in range(0,20):
    # Pull text from df
    text = corpus_new.loc[i, "Text"]
    # Run scispaCy model
    doc = nlp(text)
    # Preprocess
    ttr = type_token_ratio(doc)
    # Put in results_df
    results_df.loc[i, 'Type/Token'] = ttr

results_df.head(5)

In [ ]:
# Calculate adjusted type/token ratio (lemmatized) divided by word count

for i in range(0,20):
    # Pull text from df
    text = corpus_new.loc[i, "Text"]
    # Split by word (" " for simplicity)
    word_ct = len(text.split(' '))
    # Put in results_df
    results_df.loc[i, 'Word Count'] = word_ct
    
results_df

In [ ]:
# Divide by mean word count of all documents
results_df['TTR_adj'] = (results_df['Type/Token'] / results_df['Word Count']) * np.mean(results_df['Word Count'])
    
results_df.head(6)

In [ ]:
# Compare mean stats between JP-EN and EN-EN corpora

compare_means('Type/Token')
compare_means('Word Count')
compare_means('TTR_adj')

Comments (4/10): H1 seems to be rejected. fMRI studies authored by Japanese scientists are just as lexically sophisticated as comparable docs authored by Anglophone counterparts. Perhaps this is a good thing: i.e., any differences discovered later are a product of linguistic features, rather than scientific knowledge/ignorance.

In [ ]:
# Examine the entities extracted by the mention detector. Note that they don't have types
# like in SpaCy, and they are more general (e.g including verbs) - these are any spans
# which might be an entity in UMLS, a large biomedical database.
print(a.ents)


#>>> (Myeloid derived suppressor cells,
#     MDSC,
#     immature,
#     myeloid cells,
#     immunosuppressive activity,
#     accumulate,
#     tumor-bearing mice,
#     humans,
#     cancer,
#     hepatocellular carcinoma,
#     HCC)

In [ ]:
# We can also visualise dependency parses
# (This renders automatically inside a jupyter notebook!):
from spacy import displacy
# displacy.render(next(doc2.sents), style='dep', jupyter=True)
displacy.render(a, style='dep', jupyter=True)

# See below for the generated SVG.
# Zoom your browser in a bit!